In [2]:
import os

In [24]:
os.chdir('/Users/layne/Desktop/chest_xray/train/normal')
healthy = os.listdir('.')
healthy = [os.path.abspath(f) for f in healthy]

In [26]:
os.chdir('/Users/layne/Desktop/chest_xray/train/pneumonia')
pneumonia = os.listdir('.')
pneumonia = [os.path.abspath(f) for f in pneumonia]

In [27]:
import random
random.shuffle(healthy)
random.shuffle(pneumonia)

In [38]:
healthy_subset = healthy[:150]
pneumonia_subset = pneumonia[:150]

In [36]:
target_dir = '/Users/layne/Desktop/xray_mixed_patients'

Rename when moving over "healthy-i.jpg"

In [37]:
import shutil

In [40]:
for i, file_path in enumerate(healthy_subset):
    target_file_path = f"/Users/layne/Desktop/xray_mixed_patients/healthy-{i}.jpeg"
    shutil.copyfile(file_path, target_file_path)

In [41]:
for i, file_path in enumerate(pneumonia_subset):
    target_file_path = f"/Users/layne/Desktop/xray_mixed_patients/pneumonia-{i}.jpeg"
    shutil.copyfile(file_path, target_file_path)

In [3]:
os.chdir('/Users/layne/Desktop/xray_mixed_patients')

In [4]:
mixed = os.listdir('.')

In [5]:
import numpy as np

In [6]:
mixed_arr = []
for file in mixed:
    if 'healthy' in file:
        arr = np.array([0,file])
    elif 'pneumonia' in file:
        arr = np.array([1,file])
    mixed_arr.append(arr)

In [7]:
mixed_np = np.array(mixed_arr)

In [8]:
import pandas as pd

/Users/layne/.pyenv/versions/3.7.6/envs/jupyterlab/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [9]:
df = pd.DataFrame(mixed_np, columns=['status', 'path'])

---

In [10]:
import os
os.chdir('/Users/layne/Desktop/aiqc')

In [11]:
import aiqc
from aiqc import examples

In [12]:
aiqc.delete_db(True)


=> Success - deleted database file at path:
/Users/layne/Library/Application Support/aiqc/aiqc.sqlite3



In [13]:
from importlib import reload
reload(aiqc)

<module 'aiqc' from '/Users/layne/Desktop/aiqc/aiqc/__init__.py'>

In [14]:
aiqc.create_db()


=> Success - created database file for machine learning metrics at path:
/Users/layne/Library/Application Support/aiqc/aiqc.sqlite3


=> Success - created the following tables within database:
['algorithm', 'batch', 'datapipeline', 'dataset', 'experiment', 'featureset', 'file', 'fold', 'foldset', 'hyperparamcombo', 'hyperparamset', 'image', 'job', 'label', 'preprocess', 'result', 'splitset', 'tabular']



---

In [15]:
dataset_tab = aiqc.Dataset.Tabular.from_pandas(dataframe=df, name='xray')

In [16]:
dir_path = '/Users/layne/Desktop/xray_mixed_patients'

In [17]:
dataset_images = aiqc.Dataset.Image.from_folder(dir_path)

In [19]:
label = dataset_tab.make_label(columns=['status'])

In [20]:
featureset = dataset_images.make_featureset()

In [21]:
splitset = featureset.make_splitset(label_id=label.id, size_test=0.30)

Should refactor splitset so that it doesn't rea